<a href="https://colab.research.google.com/github/DiogoMondin/rfv-analisys/blob/main/T%26D_RFV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

import matplotlib.pyplot as plt

# Table

In [6]:
customer_dataset = "/content/customer_dataset_csv.txt"

In [7]:
spark = (
    SparkSession.builder
    .appName("RFV Analysis")
    .getOrCreate()
)

In [8]:
customer_df = (
    spark.read
    .option("header", "true")
    .option("inferschema", "true")
    .csv(customer_dataset)
)

# Data Exploring

In [9]:
customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- favorite_category: string (nullable = true)
 |-- acquisition_channel: string (nullable = true)
 |-- first_purchase_date: string (nullable = true)
 |-- total_orders: integer (nullable = true)
 |-- total_spent: integer (nullable = true)
 |-- last_purchase_date: string (nullable = true)
 |-- returns_count: integer (nullable = true)
 |-- avg_order_value: integer (nullable = true)
 |-- days_since_last_purchase: integer (nullable = true)
 |-- customer_lifetime_months: integer (nullable = true)



In [10]:
customer_df.describe().show()

+-------+-----------+-----------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+-----------------+------------------------+------------------------+
|summary|customer_id|favorite_category|acquisition_channel|first_purchase_date|     total_orders|       total_spent|last_purchase_date|     returns_count|  avg_order_value|days_since_last_purchase|customer_lifetime_months|
+-------+-----------+-----------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+-----------------+------------------------+------------------------+
|  count|         35|               35|                 35|                 35|               35|                35|                35|                35|               35|                      35|                      35|
|   mean|       NULL|             NULL|               NULL|               NULL|10.82857142857143|7965.514285

In [11]:
customer_df.show()

+-----------+-----------------+-------------------+-------------------+------------+-----------+------------------+-------------+---------------+------------------------+------------------------+
|customer_id|favorite_category|acquisition_channel|first_purchase_date|total_orders|total_spent|last_purchase_date|returns_count|avg_order_value|days_since_last_purchase|customer_lifetime_months|
+-----------+-----------------+-------------------+-------------------+------------+-----------+------------------+-------------+---------------+------------------------+------------------------+
|   CUST0001|          Tablets|             Google|         16/11/2022|          10|      17302|        25/03/2024|            0|           1730|                     124|                      20|
|   CUST0002|            Games|             Google|         18/04/2023|          21|      18160|        21/07/2024|            0|            865|                       6|                      15|
|   CUST0003|       

In [12]:
def add_rfv_cluster(df, col_name, new_col_name, reverse=False):
    df = df.withColumn(col_name, F.col(col_name).cast("int"))

    q1, q2, q3, q4 = df.approxQuantile(col_name, [0.2, 0.4, 0.6, 0.8], 0)

    if reverse:
        df = df.withColumn(
            new_col_name,
            F.when(F.col(col_name) <= q1, 1)
             .when(F.col(col_name) <= q2, 2)
             .when(F.col(col_name) <= q3, 3)
             .when(F.col(col_name) <= q4, 4)
             .otherwise(5)
        )
    else:
        df = df.withColumn(
            new_col_name,
            F.when(F.col(col_name) <= q1, 5)
             .when(F.col(col_name) <= q2, 4)
             .when(F.col(col_name) <= q3, 3)
             .when(F.col(col_name) <= q4, 2)
             .otherwise(1)
        )

    return df

In [13]:
# days_since_last_purchase → recency cluster
customer_df = add_rfv_cluster(customer_df, "days_since_last_purchase", "recency_cluster", reverse=False)

# total_orders → frequency cluster
customer_df = add_rfv_cluster(customer_df, "total_orders", "frequency_cluster", reverse=True)

# total_spent → value cluster
customer_df = add_rfv_cluster(customer_df, "total_spent", "value_cluster", reverse=True)

# all clusters

customer_df = (
    customer_df
    .withColumn("rfv_cluster", F.concat(F.col("recency_cluster"), F.col("frequency_cluster"), F.col("value_cluster")))
)

In [14]:
(
    customer_df
    .groupBy("recency_cluster")
    .agg(
        F.round(F.avg(F.col("days_since_last_purchase")), 2).alias("avg_days_since_last_purchase")
    )
    .orderBy("recency_cluster")
    .show()
)

+---------------+----------------------------+
|recency_cluster|avg_days_since_last_purchase|
+---------------+----------------------------+
|              1|                      256.29|
|              2|                       130.0|
|              3|                       55.43|
|              4|                       15.71|
|              5|                        3.57|
+---------------+----------------------------+



In [15]:
(
    customer_df
    .groupBy("frequency_cluster")
    .agg(
        F.round(F.avg(F.col("total_orders")), 2).alias("avg_total_orders")
    )
    .orderBy("frequency_cluster")
    .show()
)

+-----------------+----------------+
|frequency_cluster|avg_total_orders|
+-----------------+----------------+
|                1|             2.5|
|                2|             5.5|
|                3|             8.0|
|                4|           14.29|
|                5|           24.29|
+-----------------+----------------+



In [16]:
(
    customer_df
    .groupBy("value_cluster")
    .agg(
        F.round(F.avg(F.col("total_spent")), 2).alias("avg_total_spent")
    )
    .orderBy("value_cluster")
    .show()
)

+-------------+---------------+
|value_cluster|avg_total_spent|
+-------------+---------------+
|            1|        1311.14|
|            2|        3516.57|
|            3|        6424.71|
|            4|       10903.14|
|            5|        17672.0|
+-------------+---------------+



In [17]:
customer_clusters_df = (
    customer_df
    .withColumn(
        "tag",
        F.when(
            (F.col("days_since_last_purchase") <= 30) &
            (F.col("total_orders") >= 8) &
            (F.col("total_spent") >= 5000),
            "champions"
        ).when(
            (F.col("days_since_last_purchase") > 90) &
            (F.col("total_orders") >= 3) &
            (F.col("total_spent") >= 1000),
            "at risk"
        ).when(
            (F.col("days_since_last_purchase") <= 60) &
            (F.col("total_orders") <= 2),
            "new clients"
        ).otherwise("loyal")
    )
    .withColumn(
        "rfv_score",
        (F.col("recency_cluster") * 0.3) + (F.col("frequency_cluster") * 0.3) + (F.col("value_cluster") * 0.4)
    )
)

In [18]:
(
    customer_clusters_df
    .select(
        "days_since_last_purchase",
        "total_orders",
        "total_spent",
        "recency_cluster", "frequency_cluster", "value_cluster",
        "rfv_score",
        "tag"
    )
    .show()
)

+------------------------+------------+-----------+---------------+-----------------+-------------+------------------+-----------+
|days_since_last_purchase|total_orders|total_spent|recency_cluster|frequency_cluster|value_cluster|         rfv_score|        tag|
+------------------------+------------+-----------+---------------+-----------------+-------------+------------------+-----------+
|                     124|          10|      17302|              2|                3|            5|               3.5|    at risk|
|                       6|          21|      18160|              5|                5|            5|               5.0|  champions|
|                      88|          16|       7309|              3|                4|            3|               3.3|      loyal|
|                     101|           7|       7844|              2|                3|            4|               3.1|    at risk|
|                     323|           5|       1075|              1|                

In [19]:
(
    customer_clusters_df
    .groupBy("tag")
    .agg(
        F.round(F.avg(F.col("rfv_score")), 2).alias("avg_rfv_score")
    )
    .orderBy("avg_rfv_score")
    .show()
)

+-----------+-------------+
|        tag|avg_rfv_score|
+-----------+-------------+
|new clients|         1.75|
|    at risk|         2.17|
|      loyal|         2.74|
|  champions|         4.57|
+-----------+-------------+



# Part 2

In [20]:
(

    .groupBy("tag")
    .agg(
        F.count("customer_id").alias("clients"),
        F.sum("total_spent").alias("total_revenue"),
    )
    .withColumn(
        "avg_revenue_per_client",
        F.col("total_revenue") / F.col("clients")
    )
    .withColumn(
        "priority",
        F.when(
            F.col("tag") == "at risk",
            "high"
        )
        .when(
            F.col("tag") == "new clients",
            "medium"
        )
        .when(
            F.col("tag") == "loyal",
            "low"
        )
        .otherwise("maintenance")
    )
    .show()
)

+-----------+-------+-------------+----------------------+-----------+
|        tag|clients|total_revenue|avg_revenue_per_client|   priority|
+-----------+-------+-------------+----------------------+-----------+
|new clients|      4|         4435|               1108.75|     medium|
|      loyal|      7|        38078|     5439.714285714285|        low|
|    at risk|     13|        70382|                5414.0|       high|
|  champions|     11|       165898|    15081.636363636364|maintenance|
+-----------+-------+-------------+----------------------+-----------+



# Análise RFV e Estratégias por Segmento de Clientes

## 1. SEGMENTAÇÃO DE PERFIL

### AT RISK (Prioridade: ALTA)
- **Tamanho:** 13 clientes (36% da base)

- **Receita em risco:** `R$ 70.382` (27% do total)

- **Ticket médio:** `R$ 5.414`

- **Comportamento:** Clientes que compraram regularmente mas com desengajamento recente (baixa frequência/recência)

- **Por que priorizar:** Recuperação é 5-7x mais barata que novos clientes. Risco: perder 27% da receita.

### CHAMPIONS (Prioridade: MANUTENÇÃO)
- **Tamanho:** 11 clientes (30% da base)

- **Receita protegida:** `R$ 165.898` (64% do total)

- **Ticket médio:** `R$ 15.081` (2,8x acima da média)

- **Comportamento:** Compras frequentes e recentes, alto valor por transação

- **Por que priorizar:** 64% da receita com apenas 30% dos clientes. Perda = ROI negativo crítico.

### LOYAL (Prioridade: MÉDIA)
- **Tamanho:** 7 clientes (19% da base)

- **Receita:** `R$ 38.078`

- **Ticket médio:** `R$ 5.440`

- **Comportamento:** Padrão de compra consistente, sem risco imediato, mas abaixo de Champions

- **Por que priorizar:** Menor esforço de retenção + alta propensão a upsell. Potencial crescimento 25-40%.

### NEW CLIENTS (Prioridade: MÉDIA)
- **Tamanho:** 4 clientes (11% da base)

- **Receita:** `R$ 4.435`

- **Ticket médio:** `R$ 1.109` (39% da média)

- **Comportamento:** Primeira/segunda compra recente, ciclo de decisão em andamento

- **Por que priorizar:** Taxa de conversão para cliente retido afeta LTV. Custo de retenção menor que reativação.

---

## 2. ESTRATÉGIAS POR SEGMENTO

### SEGMENTO: AT RISK | Win-Back Campaign

**Objetivo:** Reativar 15% em 60 dias, protegendo `R$ 70.382`

**Ação Específica:**
- **Fase 1 (Semanas 1-2):** Email resgate personalizado com cupom 20% válido 7 dias, segmentado por categoria última compra. *Ex: "João, sua última compra de Games foi em agosto. 20% OFF nos lançamentos. Válido até sexta."*

- **Fase 2 (Semanas 3-4):** SMS urgência se sem abertura: "10% OFF + Frete Grátis - Termina hoje"

- **Fase 3 (Semanas 5-8):** Reengajamento relacional: webinar gratuito, conteúdo educativo (sem vendas)

**Canal:** Email 75% → SMS 20% (após 5 dias sem abertura) → App Push 5%

**Timing:** Terça-feira 14h | Máximo 2 emails + 1 SMS em 60 dias | Validade 7 dias

**Oferta:** 20% cupom categoria anterior + Frete grátis (acima `R$ 100`) + Conteúdo exclusivo

**Personalização:** Segmentar por categoria | Nome + referência última compra | Produtos similares/complementares

**Operacionalização:**
- Responsável: Marketing (automação) + Data (segmentação)

- Recursos: Mailchimp/RD Station + Designer 8h + Copywriter 6h

- Timeline: Setup 2 semanas | Execução 60 dias

- Automação: SIM - Workflow automático por inatividade (0 compras 90+ dias)

- KPI Principal: Taxa de Reativação

- Métricas Secundárias: Open Rate 22%, CTR 3,5%, AOV esperado `R$ 4.800`

**ROI:** Investimento `R$ 2.500` | Taxa sucesso 15% | 2 clientes retidos | Receita protegida `R$ 8.662` | **ROI +246% | Payback 15 dias**

---

### SEGMENTO: CHAMPIONS | VIP Retention Program

**Objetivo:** Proteger `R$ 165.898` + elevar frequência 25%

**Ação Específica:**
- Acesso antecipado a lançamentos (48h antes)

- Birthday discount 25% automático + brinde curado

- Cashback 5% acumulável (milhas/créditos)

- Atendimento telefônico VIP (suporte prioritário)

- Convite eventos exclusivos (4x/ano: Black Friday, Natal, aniversário, especial)

*Ex: "Maria, você é Nossa Campeã. Lançamento 'Game X' chega AMANHÃ só para você + 25% OFF aniversário (nov)"*

**Canal:** Email Premium (semanal) + SMS/App (semanal, exclusivo) + Telefone/Whatsapp (1x trimestral)

**Timing:** Quinta-feira 10h | 1 email/semana + 1 SMS/mês | 4 convites/ano

**Oferta:** Acesso antecipado (diferenciação) + Cashback acumulável + Presentes personalizados + Concierge

**Personalização:** Histórico categorias preferidas | Datas importantes (aniversário, anniversary) | Recomendações baseadas histórico

**Operacionalização:**
- Responsável: Customer Success + Marketing + Vendas

- Recursos: CRM + plataforma cashback + 0,5 FTE atendimento VIP

- Timeline: Setup 4 semanas | Execução permanente

- Automação: 80% automático (emails/SMS) + 20% humano (telefone)

- KPI Principal: Retention Rate (evitar churn)

- Métricas Secundárias: Frequência compra, AOV, NPS 70+, Churn Rate

**ROI:** Investimento `R$ 60.000`/ano | Taxa sucesso: evitar 2 churn/ano + 25% freq. | Receita protegida `R$ 30.162` + incremental `R$ 41.472` | **ROI +19,4% | Payback 7,5 meses | Retenção esperada 95%+**

---

### SEGMENTO: LOYAL | Upgrade Path Program

**Objetivo:** Converter 5 de 7 clientes para Champions via upsell/cross-sell, elevando AOV 30%

**Ação Específica:**
- **Fase 1:** Email "Descuberta": "Marina, você adora [Categoria]. Testou nossos kits premium? 15% OFF só para você."

- **Fase 2 (Dia 5):** SMS time-limited 48h: "Kit Premium [Categoria] 15% OFF + Brinde (`R$ 200`). Aproveita?"

- **Fase 3 (Dia 10):** Convite "Clube de Clientes" (preview VIP): acesso antecipado lançamentos

**Canal:** Email primário → SMS (se não abriu) → App Push (re-engajamento semanal)

**Timing:** Segunda-feira 11h | 1 email/semana + 1 SMS/2 semanas | Oferta válida 7 dias

**Oferta:** 15% categoria complementar (upsell) + Kit descoberta premium + Progressão VIP (gamificação)

**Personalização:** Análise histórico (Games → Acessórios Gaming) | Gap: freq. alta mas ticket baixo = upsell | Conteúdo educativo upgrade

**Operacionalização:**
- Responsável: Marketing (automação) + Product (recomendações)

- Recursos: Ferramenta recomendação + Copywriter 4h/mês

- Timeline: Setup 3 semanas | Execução contínua

- Automação: SIM - 100% via gatilhos comportamento

- KPI Principal: Taxa Conversão para Champion

- Métricas Secundárias: AOV, Open Rate 24%, CTR 4,5%

**ROI:** Investimento `R$ 9.600`/ano | Taxa sucesso 30% | 2 clientes convertidos | **Ano 1: -54% | Ano 2+: +214% (como Champions) | Payback 8 meses**

---

### SEGMENTO: NEW CLIENTS | First Repeat Purchase Campaign

**Objetivo:** Converter 75% (3 de 4) em 2ª compra em 45 dias, validando LTV

**Ação Específica:**
- **Dia 1-2:** Email agradecimento + "Guia Bem-vindo" (FAQ + dicas uso)

- **Dia 7:** Email educativo: conteúdo categoria, casos uso, vídeo

- **Dia 14:** SMS criativo: "João, primeira vez é sempre especial. PRIMEIRAEZ15 [Categoria] + Frete Grátis"

- **Dia 21:** Email social proof: reviews positivos novos clientes + 15% desconto exclusivo

- **Dia 30:** SMS final: "Desconto 15% vence em 5 dias. Aproveita?"

**Canal:** Email (educativo) → SMS (ação) + App Push (se ativo no app)

**Timing:** Primeira ação 2h após compra | Cadência dias 2, 7, 14, 21, 30 | Ofertas válidas 14 dias

**Oferta:** 15% categoria complementar + Frete grátis + Conteúdo gratuito (webinar/guia/vídeo)

**Personalização:** Recomendação cruzada por categoria | Tom amigável educativo (não vendedor)

**Operacionalização:**
- Responsável: Marketing (automação) + CS (suporte)

- Recursos: Workflow automático (setup 2h) + Copywriter 2h templates

- Timeline: Setup 2 semanas | Execução automática por novo cliente

- Automação: SIM - 100% por gatilho primeira compra

- KPI Principal: Taxa Segunda Compra (Repeat Purchase Rate)

- Métricas Secundárias: Open Rate 28%, CTR 4,2%, AOV similar ou +10%

**ROI:** Investimento `R$ 500` | Taxa sucesso 40% | 2 clientes repeat | Receita `R$ 1.774` | **ROI +254% | Payback imediato**

---

## 3. PRIORIZAÇÃO DE AÇÕES

**VITÓRIAS CERTAS** (Alto Impacto/Baixo Esforço)
- At Risk: Email automático win-back

- New Clients: Onboarding automático

**APOSTAS ESTRATÉGICAS** (Alto Impacto/Alto Esforço)
- Champions: VIP Retention Program

- Loyal: Upgrade Path automático

**EVITAR** (Baixo Impacto/Alto Esforço)
- Discagem manual personalizada

- Eventos presenciais frequentes

**Sequência Recomendada:**
1. Semana 1: New Clients onboarding (baixo custo, alto ROI)

2. Semana 2-3: At Risk win-back (proteção receita crítica)

3. Semana 4-6: Champions VIP (infraestrutura maior)

4. Semana 7+: Loyal upgrade path (sustentação crescimento)

---

## 4. RESUMO EXECUTIVO

| Segmento | Tamanho | Receita | Estratégia | Investimento | ROI Ano 1 | Ação |
|----------|---------|---------|-----------|--------------|-----------|------|
| AT RISK | 13 | `R$ 70.382` | Win-back urgência | `R$ 2.500` | +246% | IMPLEMENTAR IMEDIATO |
| CHAMPIONS | 11 | `R$ 165.898` | VIP Retention | `R$ 60.000` | +19% | IMPLEMENTAR MESES 1-2 |
| LOYAL | 7 | `R$ 38.078` | Upgrade Path | `R$ 9.600` | -54% Y1 / +214% Y2+ | IMPLEMENTAR MÊS 2-3 |
| NEW CLIENTS | 4 | `R$ 4.435` | Onboarding | `R$ 500` | +254% | IMPLEMENTAR IMEDIATO |

**Consolidado:**
- Valor total em jogo: `R$ 278.793`

- Investimento anual: `R$ 72.600`

- ROI esperado: +190% (Ano 1) + ganho recorrente retenção

- Payback consolidado: 4-5 meses

- Receita protegida/gerada: `R$ 81.902` (proteção) + `R$ 47.598` (incremental) = `R$ 129.500`